## 3.5 Multi Query Generation

In [1]:
# Step1. API KEY 설정
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
Settings.llm = OpenAI(model="gpt-4o", temperature=0.5)

In [3]:
from llama_index.core.prompts import Prompt
# 쿼리 변환용 프롬프트 설정
decompose_prompt = Prompt(
    """
    다음 [Original Query]와 관련있는 여러개의 쿼리를 3가지로 변환시켜주세요.
    출력 포맷은 아래와 같은 형태를 지켜주세요.
    
    Original Query: {query_str}
    Sub-Queries:
    1.
    2.
    3.
    """
)

In [4]:
# DecomposeQueryTransform 초기화
from llama_index.core.indices.query.query_transform.base import DecomposeQueryTransform
decompose = DecomposeQueryTransform(decompose_query_prompt=decompose_prompt, 
                                    llm=Settings.llm,
                                    verbose=True)

In [5]:
query = "RAG에 대해 알려주세요." # 입력 쿼리
query_bundle = decompose.run(query) # 쿼리 변환 수행
print(query_bundle.query_str) # 결과 출력

> Current query: RAG에 대해 알려주세요.
> New query: Original Query: RAG에 대해 알려주세요.  
Sub-Queries:  
1. RAG의 주요 기능은 무엇인가요?  
2. RAG의 장점과 단점은 무엇인가요?  
3. RAG를 사용하는 사례가 있나요?  
Original Query: RAG에 대해 알려주세요.  
Sub-Queries:  
1. RAG의 주요 기능은 무엇인가요?  
2. RAG의 장점과 단점은 무엇인가요?  
3. RAG를 사용하는 사례가 있나요?  


In [6]:
sub_query = query_bundle.query_str.split('\n')[2:]
input_query = query_bundle.query_str.split('\n')[0]
sub_query_join = '\n'.join(sub_query) # sub query 만 추출

In [7]:
from llama_index.core.prompts import PromptTemplate
final_prompt_template = PromptTemplate(
                        template = """아래 [Original-Query]와 [Sub-Queries]에 대한 내용을 모두 종합 및 요약해서 알려주세요.
                        \n{input_query}
                        \n[Sub-Queires]
                        \n{sub_query}
                        """,
                        template_vars="input_query, sub_query"
                    )
final_prompt = final_prompt_template.format(input_query=input_query,
                                            sub_query=sub_query_join)


In [8]:
print(final_prompt)

아래 [Original-Query]와 [Sub-Queries]에 대한 내용을 모두 종합 및 요약해서 알려주세요.
                        
Original Query: RAG에 대해 알려주세요.  
                        
[Sub-Queires]
                        
1. RAG의 주요 기능은 무엇인가요?  
2. RAG의 장점과 단점은 무엇인가요?  
3. RAG를 사용하는 사례가 있나요?  
                        


In [9]:
llm = OpenAI(model="gpt-4o", temperature=0.5, max_toknes=300)

### Original Query만 사용할 경우

In [10]:
## Original qeury response
response_original = llm.complete(query) # original query에 대한 답변
print(response_original)

RAG는 "Retrieval-Augmented Generation"의 약자로, 자연어 처리 분야에서 정보를 생성하는 모델에 외부 지식을 결합하여 더 정확하고 풍부한 응답을 생성하는 기술을 의미합니다. RAG는 주로 두 가지 주요 구성 요소로 이루어져 있습니다: 정보 검색기(retriever)와 생성기(generator).

1. **정보 검색기(Retriever)**: 이 단계에서는 주어진 질문이나 문맥에 맞는 관련 정보를 대규모 데이터베이스나 문서 집합에서 검색합니다. 이를 통해 모델은 외부 지식을 활용하여 더 정확한 답변을 생성할 수 있습니다.

2. **생성기(Generator)**: 검색기가 제공한 정보를 바탕으로 자연스러운 언어로 응답을 생성합니다. 이 단계는 일반적으로 Transformer 기반의 언어 모델을 사용하여 수행됩니다.

RAG는 특히 대규모 언어 모델이 훈련된 데이터에 포함되지 않은 정보에 접근해야 할 때 유용합니다. 예를 들어, 최신 뉴스나 특정 도메인에 대한 전문 지식이 필요한 경우에 효과적일 수 있습니다. 이러한 방식은 정적 모델보다 더 유연하고 업데이트하기 쉬운 시스템을 구축하는 데 도움이 됩니다.


### Multi Query를 사용할 경우

In [11]:
## Multi query response
response = llm.complete(final_prompt)

In [12]:
print(response.text)

RAG(Recursively Augmented Generation)는 자연어 처리(NLP) 분야에서 정보를 검색하고 생성하는 데 사용되는 기술입니다. 주로 대규모 언어 모델과 정보 검색 시스템을 결합하여 질문에 대한 정확하고 풍부한 답변을 생성하는 데 활용됩니다.

1. **RAG의 주요 기능:**
   - **정보 검색:** RAG는 대규모 데이터베이스나 문서 집합에서 관련 정보를 검색하는 기능을 가지고 있습니다. 이를 통해 질문에 대한 배경 정보를 수집합니다.
   - **텍스트 생성:** 검색된 정보를 기반으로 자연어 텍스트를 생성합니다. 이는 대화형 AI나 질문 답변 시스템에서 사용될 수 있습니다.
   - **정보 통합:** 검색된 정보와 생성된 텍스트를 통합하여 사용자에게 더 풍부한 답변을 제공합니다.

2. **RAG의 장점과 단점:**
   - **장점:**
     - **정확성 향상:** 검색된 정보에 기반하여 답변을 생성하기 때문에 일반적인 언어 모델보다 더 정확한 답변을 제공할 수 있습니다.
     - **정보 다양성:** 다양한 출처에서 정보를 수집하여 더 다양한 관점을 제공할 수 있습니다.
   - **단점:**
     - **복잡성:** 시스템의 복잡성이 증가하여 구현 및 유지보수가 어려울 수 있습니다.
     - **실시간성 문제:** 대규모 데이터베이스에서 정보를 검색하고 처리하는 데 시간이 걸릴 수 있어 실시간 응답이 어려울 수 있습니다.

3. **RAG를 사용하는 사례:**
   - **고객 지원:** 고객의 질문에 대한 정확한 답변을 제공하기 위해 RAG를 활용하는 사례가 있습니다. 이를 통해 고객 만족도를 높일 수 있습니다.
   - **의료 정보 제공:** 의료 분야에서 환자나 의료진에게 정확한 정보를 제공하기 위해 RAG를 사용할 수 있습니다.
   - **학술 연구:** 연구자들이 방대한 양의 학술 문헌에서 필요한 정보를 빠르게 검색하고 요약하는 데 도움을 줄 수 있습니다.

RAG는 정보 검색과 생성의 장점을